# Imports

In [1]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict

# Task 1: Vocabulary Creation 

### Read train.json

In [2]:
with open('data/train.json') as f:
    train_data = json.load(f)

### Count occurence of every word

In [3]:
vocab_dict = {}
for train in train_data:
    for word in train['sentence']:
        vocab_dict[word] = 1 + vocab_dict.get(word, 0)

### Count unknown words

In [4]:
threshold = 3
unknown_count = 0
for key in vocab_dict.copy():
    if vocab_dict[key] < threshold:
        unknown_count += vocab_dict[key]
        del vocab_dict[key]

### Sort based on occurence

In [5]:
sorted_vocab = sorted(vocab_dict.items(), key=lambda x:x[1], reverse=True)

### Create vocab.txt

In [6]:
index = 1
with open('vocab.txt', 'w') as f:
    f.write('unk\t{}\t{}\n'.format(index, unknown_count))
    index += 1
    for key, val in sorted_vocab:
        f.write('%s\t%s\t%s\n' % (key, index, val))
        index += 1

# Task 2: Model Learning

#### Reading Training data and vocab.txt

In [7]:
with open('data/train.json', 'r') as train_file:
    training_data = json.load(train_file)

# Reading vocab.txt to determine unknown words
vocab_data = []
with open('vocab.txt', 'r') as vocab_file:
    for line in vocab_file:        
        words = line.rsplit()        
        if len(words)>2:
            vocab_data.append(words[0])        

### Generating transition and emission parameters

In [8]:
transition, emission, label_dict = {}, {}, {}
unique_label = set()

for record in training_data:
    labels = record['labels']
    sentence = record['sentence']

    for i in range(len(labels)):
        label = labels[i]
        word = sentence[i]

        unique_label.add(label)
        label_dict[label] = 1 + label_dict.get(label, 0)

        if word.isdigit():
            word = '<isdigit>'

        if word not in vocab_data:
            word = '<unk>'

        emission[(label, word)] = 1 + emission.get((label, word), 0)

        if i == 0:
            transition[('.', label)] = 1 + transition.get(('.', label), 0)

        else:
            prev_label = labels[i-1]
            transition[(prev_label, label)] = 1 + transition.get((prev_label, label), 0)


In [9]:
label_total = {}
for t in transition:
    label_total[t[0]] = transition[t] + label_total.get(t[0], 0)

#### Performing laplace_smoothing to remove any 0 probability from the transition parameter

In [10]:
def laplace_smoothing(transition_dict_counts, k, states, label_total):
    trans_probs = {}
    for current_state in states:
        for next_state in states:
            if (current_state,next_state) in transition_dict_counts:
                tg=transition_dict_counts[current_state,next_state]
            else:
                tg=0
            trans_probs[current_state, next_state] = ( tg + k) / (label_total[current_state] + k * len(states))
    return trans_probs

transition = laplace_smoothing(transition, 1, list(unique_label), label_total)

### Emission Parameter

In [11]:
for i in emission:
    emission[i] /= label_dict[i[0]]

### Generating hmm.json

In [12]:
transition_parameter = {}
for key, val in transition.items():    
    k1, k2 = key[0], key[1]
    transition_parameter[str(k1)+","+str(k2)] = val

emission_parameter = {}
for key, val in emission.items():    
    k1, k2 = key[0], key[1]
    emission_parameter[str(k1)+","+str(k2)] = val

hmm = {}
hmm['transition'] = transition_parameter
hmm['emission'] = emission_parameter

with open('hmm.json', 'w') as json_file:
    json.dump(hmm, json_file, indent=4, sort_keys=True)    

### How many transition and emission parameters in your HMM?

In [13]:
with open('hmm.json', 'r') as json_file:
    data = json.load(json_file)

print("Number of transition parameters", len(data['transition']))
print("Number of emission parameters", len(data['emission']))

Number of transition parameters 2025
Number of emission parameters 23015


# Task 3: Greedy Decoding with HMM

In [14]:
with open('data/dev.json', 'r') as dev_file:
    dev_data = json.load(dev_file)

dev_words = []
for dev in dev_data:
    for word in dev['sentence']:
        dev_words.append(word)

In [15]:
unknown_words = {}
for word in train['sentence']:
    if word not in sorted_vocab:
        unknown_words[word] = 1 + unknown_words.get(word, 0)

In [16]:
def greedy_decoding(words, unique_label):
    res = []
    all_state = []
    T = list(unique_label)
    k = 1

    for key, word in enumerate(words):        
        t = ""
        max_pr = 0

        if word not in vocab_data:
            res.append((word, 'NNP'))
            all_state.append('NNP')
            continue

        for tag in T:
            stateval = 0
            if key == 0:
                trans_val = transition[('.', tag)]
            else:
                trans_val = transition[(all_state[-1], tag)]

            for z in emission:
                if z[1] == word and z[0] == tag:
                    yi = trans_val*emission[z]
                    stateval += yi

            if stateval >= max_pr:
                max_pr = stateval
                t = tag
                
        all_state.append(t)
        res.append((word, t))

        k=k+1
        if k%500==0:
          print(k)

    return res    

In [17]:
res = greedy_decoding(dev_words, unique_label)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [18]:
ground_truth = []
for entry in dev_data:
    sentence = entry["sentence"]
    labels = entry["labels"]
    pairs = list(zip(sentence, labels))
    ground_truth.extend(pairs)

In [19]:
correct_predictions = 0
wrong_predictions = 0

# Iterate through both lists and compare predictions
for (wordA, labelA), (wordB, labelB) in zip(res, ground_truth):
    if labelA == labelB:
        correct_predictions += 1
    else:
        wrong_predictions += 1

In [20]:
print("correct",(correct_predictions/(wrong_predictions+correct_predictions))*100)
print("incorrect",(wrong_predictions/(wrong_predictions+correct_predictions))*100)

correct 92.25456863578411
incorrect 7.745431364215895


### Predicting on Test Data

In [21]:
with open('data/test.json', 'r') as test_file:
    test_data = json.load(test_file)

test_words = []
for test in test_data:
    for word in test['sentence']:
        test_words.append(word)

In [22]:
test_res = greedy_decoding(test_words, unique_label)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


### Generating greedy.json

In [23]:
with open('data/test.json', 'r') as test_file:
    greedy_data = json.load(test_file)

In [35]:
k = 0
for i in greedy_data:
    test_labels = []
    for word in i['sentence']:
        if test_res[k][0] == word:
            test_labels.append(test_res[k][1])
        k+=1
    i['labels'] = test_labels

In [37]:
output_file = 'greedy.json'
with open(output_file, "w") as outfile:
    json.dump(greedy_data, outfile, indent=4)

# Task 4: Viterbi Decoding with HMM

In [26]:
with open('data/dev.json', 'r') as dev_file:
    dev_data = json.load(dev_file)

dev_sentences = []
for index, dev in enumerate(dev_data):
    current_sentence = []
    for word in dev['sentence']:
        current_sentence.append(word)
    dev_sentences.append(current_sentence)


dev_sentences2 = []
for index, dev in enumerate(dev_data):
    current_sentence = []
    for index, word in enumerate(dev['sentence']):
        dt=word+'/'+ dev['labels'][index]
        current_sentence.append(dt)
    dev_sentences2.append(current_sentence)


In [28]:
NN_SUFFIX = ["action", "age", "ance", "cy", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling",
               "ment", "ness", "or", "ry", "scape", "ship", "dom", "ty"]
VB_SUFFIX = ["ed", "ify", "ise", "ize", "ate", "ing"]
JJ_SUFFIX = ["ous", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "able","wise"]
ADV_SUFFIX = ["ward"]
VBG_suffix="ing"
VBN_suffix="ed"
NNS_suffix=["s","ies","wards","es"]

# MORPHOLOGY RULES
def morph_rules(word):
    if any(char.isdigit() for char in word):
        if word.startswith('$'):
            return "CD"
        return "CD"
    elif any(char.isupper() for char in word):
        return "NNP"
    elif any(word.endswith(suffix) for suffix in NN_SUFFIX):
        return "NN"
    elif any(word.endswith(suffix) for suffix in VB_SUFFIX):
        return "VB"
    elif any(word.endswith(suffix) for suffix in JJ_SUFFIX):
        return "JJ"
    elif any(word.endswith(suffix) for suffix in ADV_SUFFIX):
        return "RB"
    elif any(word.endswith(suffix) for suffix in NNS_suffix):
        return "NNS"
    elif word.endswith("ing"):
        return "VBG"
    elif word.endswith("ed"):
        return "VBN"
    elif word.istitle():
        return "NNP"
    elif word.endswith("'s"):
        return"POS"
    elif '-' in word:
        return "JJ"
    return "NNP"

In [29]:
def viterbi2(sentence_list):
        result=[]
        jk=1
        
        for index,sentences in enumerate(sentence_list):
            # since the tags are being read from the set of tags present in train, it is important to verify whether the tag, word pair exist in 
            # training data, hence a check has been applied everytime a new tag is considered for the word and in case of absence, the probability is 
            # assigned as 0.
            # Similarly, there are pairs of tags that are not present in the training data A check is applied for verifying the the presence of the same 
            # in the transition dictionary.

            prob_values, prob_values_bp = list(), list()
            for wno, word in enumerate(sentences):

                # There are three conditions- for every conditions the following process is followed:
                # 1. The tag,word pair is verified to be in emission dictionary, and the emission probability is extracted
                # 2. The tag is considered with the previous tag i.e the tag assigned to the previous word and the transition probability is calculated.
                # 3. The current word probability is calulated using the emission and transition probabilities
                # 4. The tag with the best current probability is assigned to the state in the back propagation list, which will then be considered in the future and
                # the probability of that tag is included in the forward propagation list which is used in current probability calculation of previous states.

                
                
                prob_values.append({})
                prob_values_bp.append({})

                # Condition to consider numbers, the numbers were renames as digits, because there can is a very large set of numbers that can occur, 
                # but all have to be treated as a digit, so whenever isdigits() is true, the pair of tags with teh term "digit" is looked up in the
                # emission matrix. 

                if(wno !=0 and word.isdigit()):
                    for t2 in unique_label:
                        # the probability of each tag, word pair is set to be a random negative value which is very low, as probability will always be 
                        # positive, and this way we can confirm whetehr the tag,word pair has been considered in the forward propagation
                        prob_values[wno][t2] = -1000000
                        if (t2,'<isdigit>') in emission:
                            curr_emission = emission[t2,'<isdigit>']
                        else:
                            curr_emission=0

                        for t1 in prob_values_bp[wno-1]:
                            if (t1, t2) in transition:
                                curr_transition = transition[t1, t2]
                            else:
                                curr_transition=0
                            curr_prob = curr_emission*curr_transition
                            if(prob_values[wno][t2] < curr_prob):
                                prob_values[wno][t2] = curr_prob
                                prob_values_bp[wno][t2] = t1

                # The first word of sentences generally come after full stops. hence instead of using a start tag, I have considered transition from 
                # full stop to other tags in case of firts words.
                elif(wno == 0):
                    for t2 in unique_label:
                        if (t2,word) in emission:
                            et= emission[t2,word]
                        else:
                            et=0
                        if ('.',t2) in transition:
                            tt=transition['.',t2]
                        else:
                            tt=0
                        prob_values[wno][t2] = et * tt
                        prob_values_bp[wno][t2] = '.'

                else:
                    # the temporary flag is used to verify whether the word is part of the training data vocabulary or not. 
                    # if a non zero emission probability is not determined for the tag, word- the word is considered an unknown word

                    flag = 1
                    for t2 in prob_values[wno-1]:
                        # the probability of each tag, word pair is set to be a random negative value which is very low, as probability will always be 
                        # positive, and this way we can confirm whetehr the tag,word pair has been considered in teh forward propagation
                        prob_values[wno][t2] = -1000000
                        if (t2,word) in emission:
                            curr_emission=emission[t2,word]
                        else:
                            curr_emission=0
                        
                        # if the tag,word pair exist then only we calculate the current state probability, if it does not exist, we skip the state probability and 
                        # follow the steps considering the word as 'unk'
                        if(curr_emission != 0):
                            flag = 0

                            for t1 in prob_values_bp[wno-1]:
                                if (t1, t2) in transition:
                                    tt = transition[t1,t2]
                                else:
                                    tt=0
                                curr_prob = tt*curr_emission*prob_values[wno-1][t1]

                                if(prob_values[wno][t2] < curr_prob):
                                    prob_values[wno][t2] = curr_prob
                                    prob_values_bp[wno][t2] = t1

                    if(flag):
                        # if the word is considered as an unknown word, the occurence of tag with the keyword 'unk' is considered.
                        # based on the vocabulary created using training data, words occurring less than 3 times have been replaced with the 'unk' keyword.
                        # the tag for the uknown words were determined using predefined morphological rules to determine type of word based on the structure of word. 
                        # Also, based on research, it was determined that one of the ways to deal with unknown words is to assign the tag which occurs the most
                        # along the unknown words of the training dictionary. 
                        # The most occuring tag,'unk pair was found to be with the NNP tag. Hence, if the tag,'unk' does not exist in the emission dictionary, 
                        # the value of NNP,'unk' is obtained from the emission dictionary and used.
                            t2= morph_rules(word)
                            for t1 in prob_values_bp[wno-1]:
                                if (t2,'<unk>') in emission:
                                    et=emission[t2,'<unk>']
                                else:
                                    et=emission['NNP','<unk>']
                                if (t1, t2) in transition:
                                    tt = transition[t1,t2]
                                else:
                                    tt=0

                                curr_prob = et * tt * prob_values[wno-1][t1]

                                if(prob_values[wno][t2] < curr_prob):
                                    prob_values[wno][t2] = curr_prob
                                    prob_values_bp[wno][t2] = t1

            
            for tag in prob_values_bp[wno]:
                if (tag,'.') in transition:
                    lprob= transition[tag,'.']
                else:
                    lprob=0
                prob_values[wno][tag] += lprob

            best_tag = max(prob_values[-1], key=prob_values[-1].get)

            tagged_sentence = list()
            for i in range(len(prob_values)-1, -1, -1): 
                tagged_words = sentences[i]+'/'+ best_tag
                tagged_sentence.append(tagged_words)
                best_tag = prob_values_bp[i][best_tag]
            
            left = 0
            right = len(tagged_sentence)-1
            while (left < right):
                # Swap
                temp = tagged_sentence[left]
                tagged_sentence[left] = tagged_sentence[right]
                tagged_sentence[right] = temp
                left += 1
                right -= 1
            result.append(tagged_sentence)
        return result

In [30]:
vit_full4=viterbi2(dev_sentences)

In [31]:
vit_full4

[['The/DT',
  'Arizona/NNP',
  'Corporations/NNS',
  'Commission/NNP',
  'authorized/VBD',
  'an/DT',
  '11.5/CD',
  '%/NN',
  'rate/NN',
  'increase/NN',
  'at/IN',
  'Tucson/NNP',
  'Electric/NNP',
  'Power/NNP',
  'Co./NNP',
  ',/,',
  'substantially/RB',
  'lower/JJR',
  'than/IN',
  'recommended/JJ',
  'last/JJ',
  'month/NN',
  'by/IN',
  'a/DT',
  'commission/NN',
  'hearing/NN',
  'officer/NN',
  'and/CC',
  'barely/RB',
  'half/PDT',
  'the/DT',
  'rise/NN',
  'sought/VBN',
  'by/IN',
  'the/DT',
  'utility/NN',
  './.'],
 ['The/DT',
  'ruling/NN',
  'follows/VBZ',
  'a/DT',
  'host/NN',
  'of/IN',
  'problems/NNS',
  'at/IN',
  'Tucson/NNP',
  'Electric/NNP',
  ',/,',
  'including/VBG',
  'major/JJ',
  'write-downs/NNS',
  ',/,',
  'a/SYM',
  '60/POS',
  '%/NN',
  'slash/VBP',
  'in/RBR',
  'the/VBP',
  'common/NN',
  'stock/VB',
  'dividend/NN',
  'and/NNP',
  'the/VBP',
  'departure/NN',
  'of/RP',
  'former/NN',
  'Chairman/NNP',
  'Einar/NNP',
  'Greve/NNP',
  'during/IN'

In [32]:
def accurracy_cal(sentence_list,vit):
    data1=[]
    for i in range(len(sentence_list)):
        for x in sentence_list[i]:
            i=x.split('/')
            data1.append(i)
    data2=[]
    for i in range(len(vit)):
        for x in vit[i]:
            i=x.split('/')
            data2.append(i)
    correct=0
    incorrect=0
    for i in range(len(data2)):
        if data2[i][0]==data1[i][0]:
            if data2[i][1]==data1[i][1]:
                correct=correct+1
            else:
                incorrect=incorrect+1
    t=correct+incorrect
    return correct/t, incorrect/t


accurracy_cal(dev_sentences2,vit_full4)

(0.8569227733592375, 0.14307722664076256)